Tutorial found [here, in the pyTables web site](http://www.pytables.org/usersguide/tutorials.html)

In this notebook, we gonna learn:
* how to save a 3D array into a HDF5 file
* load an array from a HDF5 file


In [5]:
import tables as tb
import numpy as np

# Creating a fake 3D data set

In [6]:
stack_images = []
for i in range(10):
    _random_image = np.random.rand(100,100)
    stack_images.append(_random_image)

In [7]:
print(np.shape(stack_images))

(10, 100, 100)


# Saving 3D array into h5 file

In [8]:
output_file = 'my_hdf5_file.h5'

In [9]:
h5file = tb.open_file(output_file, mode='w', title='Test file')

Creating a group

In [ ]:
group = h5file.create_group("/", "MCP", "MCP data set")

creating array object

In [12]:
h5file.create_array(group, "data", stack_images, "3D data set")

/MCP/data (Array(10, 100, 100)) '3D data set'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'python'
  byteorder := 'little'
  chunkshape := None

In [13]:
h5file.close()

# Loading array for h5 file 

In [14]:
input_file = output_file

In [15]:
new_h5file = tb.open_file(input_file, 'a')

In [16]:
print(new_h5file)

my_hdf5_file.h5 (File) 'Test file'
Last modif.: 'Wed Oct 18 12:10:12 2017'
Object Tree: 
/ (RootGroup) 'Test file'
/MCP (Group) 'MCP data set'
/MCP/data (Array(10, 100, 100)) '3D data set'



In [18]:
group = new_h5file.get_node("/MCP/data") 

In [20]:
data = group.read()

In [24]:
print(np.shape(data))
np_data = np.array(data)

(10, 100, 100)


In [25]:
print(type(np_data))

<class 'numpy.ndarray'>
